# Dashboard (SQL & Tableau)

## Import Libraries and Files

In [9]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.


In [10]:
import duckdb

# Load CSV into DuckDB
con = duckdb.connect()
con.execute("CREATE TABLE train_df AS SELECT * FROM read_csv_auto('/Users/yen/Desktop/instacart-reorder-prediction/notebooks/train_df.csv')")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Top20 Department - by order

In [16]:
top20_department_order = con.execute("""
    SELECT 
        department, 
        COUNT(*) AS total_orders
    FROM train_df
    GROUP BY department
    ORDER BY total_orders DESC
    LIMIT 20
""").df()

top20_department_order.head()

,department,total_orders
0,produce,6038459
1,dairy eggs,3451649
2,snacks,1830236
3,beverages,1714607
4,frozen,1424216


In [20]:
top20_department_order.to_csv("/Users/yen/Desktop/instacart-reorder-prediction/notebooks/top20_department_order.csv", index=False)

## Top20 Department - by reorder rate

In [17]:
top20_department = con.execute("""
    SELECT 
        department,
        SUM(reordered)::FLOAT / COUNT(*) AS reorder_rate,
        COUNT(*) AS total_orders
    FROM train_df
    GROUP BY department
    ORDER BY reorder_rate DESC
    LIMIT 20
""").df()

top20_department.head()

,department,reorder_rate,total_orders
0,dairy eggs,0.670183,3451649
1,beverages,0.654107,1714607
2,produce,0.649622,6038459
3,bakery,0.627127,747555
4,deli,0.607713,669536


In [21]:
top20_department.to_csv("/Users/yen/Desktop/instacart-reorder-prediction/notebooks/top20_department.csv", index=False)

## Top20 Reordered Product - by reorder rate

In [13]:
top20_product = con.execute("""
    SELECT 
        product_id,
        product_name,
        SUM(reordered)::FLOAT / COUNT(*) AS reorder_rate,
        COUNT(*) AS total_orders
    FROM train_df
    GROUP BY product_id, product_name
    ORDER BY reorder_rate DESC
    LIMIT 20
""").df()

top20_product.head()

,product_id,product_name,reorder_rate,total_orders
0,700,Jet Fuel Dark Roast Whole Bean Coffee,0.961538,26
1,2075,Serenity Ultimate Extrema Overnight Pads,0.960526,76
2,39992,"Energy Shot, Grape Flavor",0.952381,21
3,27740,Chocolate Love Bar,0.945652,92
4,14361,Honey Sunflower Butter,0.944444,18


In [22]:
top20_product.to_csv("/Users/yen/Desktop/instacart-reorder-prediction/notebooks/top20_product.csv", index=False)

## Peak - hour of day

In [18]:
peak_hour = con.execute("""
    SELECT 
        order_hour_of_day,
        COUNT(*) AS order_count
    FROM train_df
    GROUP BY order_hour_of_day
    ORDER BY order_hour_of_day
""").df()

print(peak_hour)

    order_hour_of_day  order_count
0                   0       141526
1                   1        73752
2                   2        45161
3                   3        33702
4                   4        34559
5                   5        56754
6                   6       185751
7                   7       569718
8                   8      1092650
9                   9      1559997
10                 10      1754849
11                 11      1740173
12                 12      1667360
13                 13      1693221
14                 14      1709262
15                 15      1696152
16                 16      1614938
17                 17      1332578
18                 18      1044301
19                 19       808658
20                 20       621007
21                 21       509382
22                 22       401732
23                 23       254808


In [23]:
peak_hour.to_csv("/Users/yen/Desktop/instacart-reorder-prediction/notebooks/peak_hour.csv", index=False)

## Peak - day of week

In [19]:
peak_day = con.execute("""
    SELECT 
        order_dow,
        SUM(reordered)::FLOAT / COUNT(*) AS reorder_rate,
        COUNT(*) AS total_orders
    FROM train_df
    GROUP BY order_dow
    ORDER BY order_dow
""").df()

print(peak_day)

   order_dow  reorder_rate  total_orders
0          0      0.584850       3958392
1          1      0.603684       3601985
2          2      0.589748       2681752
3          3      0.585919       2438791
4          4      0.591107       2406032
5          5      0.595252       2687095
6          6      0.573788       2867944


In [24]:
peak_day.to_csv("/Users/yen/Desktop/instacart-reorder-prediction/notebooks/peak_day.csv", index=False)

## Customer Segment

In [ ]:
import pandas as pd
train_df = pd.read_csv('/Users/yen/Desktop/instacart-reorder-prediction/notebooks/train_df.csv')
user_reorder = train_df.groupby('user_id')['reordered'].mean().reset_index()
user_reorder.columns = ['user_id', 'user_reorder_rate']

user_reorder.to_csv('/Users/yen/Desktop/instacart-reorder-prediction/notebooks/user_reorder_rate.csv', index=False)